# The Battle of Neighborhoods
### Opening a restaurant in Lyon
## Description of the problem

Lyon is one of the biggest French cities. Located inside the country it has two rivers flowing inside: the Saône and the Rhône which joins to become the Rhône. In addition, this city has many hills which gives it a unique look.



Inside the city center are many restaurant, cafés and bars, especially on the “presqu’île” (area between the two rivers before they merge). At night the city is crowded by people looking to have a good time. There are many different types of restaurant, the most typical being the “bouchon Lyonnais”.


I’ve got a friend who would like to open a restaurant in the city. It would be a restaurant of American food since he is born in the US. He also believes that American food restaurant are not widespread in this area which would give his business a competitive advantage.
However, he is unsure of where to install his business inside the city. 


The goal of this project is then to give him a set of different places where he could install his new restaurant as well as a set of different indicators helping him to understand the advantages of each area in order to adapt his offer. (attendance, number of restaurants installed, tourism activity, …).

## Description of the data and how to use it

I will use the Foursquare data in this project. It contains a certain number of restaurants located in Lyon. For each restaurant we have access to different type of information, namely the type of food, the price, their score, reviews etc…



I will retrieve the data using the methods learned in this course. I will then try to identify different areas using clustering techniques. Having these areas, I will then try to determine specific characteristics for each using different indicators and charts.

Once those areas and characteristics are defined, I will make a selection to determine which areas are the best to install a new restaurant.
This will be the result I will hand to my friend so he can investigate those areas by himself.